## Ajuste dos conjuntos de dados

In [1]:
# libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

In [7]:
# Vendas
df_vendas = pd.read_parquet('fato_vendas_varejo.parquet')

# Filias

df_filiais = pd.read_parquet('filiais.parquet')

# Metas Vendas

df_metas_vendas = pd.read_parquet('metas_vendas_varejo.parquet')

In [8]:
# foi identificado que a filial 1704 não existe no data set de vendas e metas de vendas
# entao optei pela remoção

filial_remover = 1704

df_vendas = df_vendas[df_vendas["codigo_filial"] != filial_remover].copy()
df_metas_vendas = df_metas_vendas[df_metas_vendas["codigo_filial"] != filial_remover].copy()

#print("df_vendas filiais:", df_vendas["codigo_filial"].nunique())
#print("df_metas_vendas filiais:", df_metas_vendas["codigo_filial"].nunique())
#print("df_filiais filiais:", df_filiais["codigo_filial"].nunique())

### Ajuste data set **Vendas**

In [25]:
df_vendas["data"] = pd.to_datetime(df_vendas["data_emissao"]).dt.normalize()

vendas_dia = (
    df_vendas
    .groupby(["codigo_filial", "data"], as_index=False)
    .agg(
        quantidade=("quantidade", "sum"),
        faturamento=("faturamento", "sum"),
        n_vendas=("codigo_documento_saida", "nunique"),  # qtd de vendas no dia
    )
    .sort_values(["codigo_filial", "data"])
    .reset_index(drop=True)
)

### Ajuste data set **Metas Vendas**

In [26]:
df_metas_vendas["data"] = pd.to_datetime(df_metas_vendas["data_meta_venda"]).dt.normalize()

df_metas_vendas = (
    df_metas_vendas
    .sort_values(["codigo_filial", "data"])
    .reset_index(drop=True)
)

### Ajuste data set **Filiais**

In [18]:
df_filiais["codigo_filial"] = df_filiais["codigo_filial"].astype(int)

## Alguma verificações de inconsistências

In [19]:
print("dtype vendas_dia:", vendas_dia["codigo_filial"].dtype)
print("dtype df_filiais:", df_filiais["codigo_filial"].dtype)
print("dtype df_filiais:", df_metas_vendas["codigo_filial"].dtype)

dtype vendas_dia: int64
dtype df_filiais: int64
dtype df_filiais: int64


In [27]:
vendas_dia

,codigo_filial,data,quantidade,faturamento,n_vendas
0,1500,2025-01-02,828.0,18077.83704,138
1,1500,2025-01-03,1107.0,29631.50298,188
2,1500,2025-01-04,597.0,14137.64100,98
3,1500,2025-01-06,975.0,21558.72465,166
4,1500,2025-01-07,969.0,25274.68470,165
...,...,...,...,...,...
42186,1887,2025-12-27,597.0,17850.20877,99
42187,1887,2025-12-28,291.0,9301.56894,62
42188,1887,2025-12-29,537.0,17140.43700,88
42189,1887,2025-12-30,735.0,16174.26540,106


In [28]:
df_metas_vendas

,codigo_filial,data_meta_venda,meta_n_med,data
0,1500,2025-01-01 03:00:00.000,0.00,2025-01-01
1,1500,2025-01-02 03:00:00.000,12732.76,2025-01-02
2,1500,2025-01-03 03:00:00.000,12732.76,2025-01-03
3,1500,2025-01-04 03:00:00.000,12732.76,2025-01-04
4,1500,2025-01-05 03:00:00.000,0.00,2025-01-05
...,...,...,...,...
42648,1887,2025-12-27 03:00:00.000,15986.05,2025-12-27
42649,1887,2025-12-28 03:00:00.000,16638.55,2025-12-28
42650,1887,2025-12-29 03:00:00.000,16312.30,2025-12-29
42651,1887,2025-12-30 03:00:00.000,14681.07,2025-12-30


In [42]:
A = vendas_dia.copy()
B = df_metas_vendas.copy()

ID_A, DT_A = "codigo_filial", "data"
ID_B, DT_B = "codigo_filial", "data"   # usa a coluna 'data' já “limpa”
META = "meta_n_med"

# 1) padroniza datas
A[DT_A] = pd.to_datetime(A[DT_A]).dt.date
B[DT_B] = pd.to_datetime(B[DT_B]).dt.date

print("Tamanhos:", len(A), len(B))
print("Filiais:", "A =", A[ID_A].nunique(), "| B =", B[ID_B].nunique())

# 2) duplicidade por chave (filial,data)
dupA = A.duplicated([ID_A, DT_A]).sum()
dupB = B.duplicated([ID_B, DT_B]).sum()
print("\nDuplicados (filial,data): A =", dupA, "| B =", dupB)

if dupA:
    print("\nExemplos duplicados A:")
    print(A[A.duplicated([ID_A, DT_A], keep=False)].sort_values([ID_A, DT_A]).head(10))

if dupB:
    print("\nExemplos duplicados B:")
    print(B[B.duplicated([ID_B, DT_B], keep=False)].sort_values([ID_B, DT_B]).head(10))

# 3) conjuntos de chaves
keysA = set(zip(A[ID_A], A[DT_A]))
keysB = set(zip(B[ID_B], B[DT_B]))

only_in_B = keysB - keysA   # meta existe, venda não existe (loja não abriu / A não registrou)
only_in_A = keysA - keysB   # venda existe, meta não existe (sab. cadastral / exceção)

print("\nChaves só no B (meta sem venda):", len(only_in_B))
print("Chaves só no A (venda sem meta):", len(only_in_A))

onlyB_df = pd.DataFrame(list(only_in_B), columns=[ID_A, DT_A])
onlyA_df = pd.DataFrame(list(only_in_A), columns=[ID_A, DT_A])

# 4) top filiais e datas que explicam a diferença
print("\nTop filiais com mais 'meta sem venda':")
print(onlyB_df.groupby(ID_A).size().sort_values(ascending=False).head(25))

print("\nTop datas com mais 'meta sem venda':")
print(onlyB_df.groupby(DT_A).size().sort_values(ascending=False).head(25))

# 5) agora: separa 'meta sem venda' em meta=0 vs meta>0 (importante!)
B_key = B[[ID_B, DT_B, META]].drop_duplicates([ID_B, DT_B])
onlyB_with_meta = onlyB_df.merge(B_key, left_on=[ID_A, DT_A], right_on=[ID_B, DT_B], how="left")

print("\nBreakdown 'meta sem venda' por meta=0 vs meta>0:")
print((onlyB_with_meta[META] == 0).value_counts().rename(index={True:"meta=0", False:"meta>0"}))

print("\nTop datas (meta=0) sem venda:")
print(onlyB_with_meta[onlyB_with_meta[META] == 0].groupby(DT_A).size().sort_values(ascending=False).head(15))

print("\nTop datas (meta>0) sem venda [mais crítico]:")
print(onlyB_with_meta[onlyB_with_meta[META] > 0].groupby(DT_A).size().sort_values(ascending=False).head(15))

# 6) merge diagnóstico (pra você “ver” o motivo em uma tabela)
diag = A.merge(
    B[[ID_B, DT_B, META]].drop_duplicates([ID_B, DT_B]),
    left_on=[ID_A, DT_A],
    right_on=[ID_B, DT_B],
    how="outer",
    indicator=True
)

print("\nDiagnóstico merge (_merge):")
print(diag["_merge"].value_counts())

print("\nExemplos: meta sem venda (right_only):")
print(diag[diag["_merge"] == "right_only"].sort_values([ID_B, DT_B]).head(10))

print("\nExemplos: venda sem meta (left_only):")
print(diag[diag["_merge"] == "left_only"].sort_values([ID_A, DT_A]).head(10))


Tamanhos: 42191 42653
Filiais: A = 124 | B = 124

Duplicados (filial,data): A = 0 | B = 0

Chaves só no B (meta sem venda): 522
Chaves só no A (venda sem meta): 60

Top filiais com mais 'meta sem venda':
codigo_filial
1749    62
1509    58
1548    57
1590    55
1500    55
1734    52
1578    40
1860    27
1854    26
1845    18
1857    18
1875    16
1617    13
1887     3
1713     2
1716     2
1545     2
1629     2
1698     2
1701     2
1656     2
1647     2
1560     1
1527     1
1635     1
dtype: int64

Top datas com mais 'meta sem venda':
data
2025-12-25    20
2025-01-01    15
2025-01-05     8
2025-01-19     8
2025-02-02     8
2025-04-13     8
2025-05-18     8
2025-08-24     8
2025-05-25     8
2025-09-28     8
2025-08-31     8
2025-09-07     8
2025-05-11     8
2025-04-06     8
2025-04-20     8
2025-04-27     8
2025-05-04     8
2025-02-23     8
2025-02-16     8
2025-02-09     8
2025-01-12     8
2025-09-21     8
2025-10-05     8
2025-10-19     8
2025-10-12     8
dtype: int64

Breakdown 'm

In [43]:
# A = vendas_dia.copy()
# B = df_metas_vendas.copy()

# A["data"] = pd.to_datetime(A["data"]).dt.date
# B["data"] = pd.to_datetime(B["data"]).dt.date

# base = A.merge(
#     B[["codigo_filial", "data", "meta_n_med"]].drop_duplicates(["codigo_filial", "data"]),
#     on=["codigo_filial", "data"],
#     how="outer",
#     indicator=True
# )

# # completa vendas ausentes com 0 (quando não existe linha no A)
# for col in ["quantidade", "faturamento", "n_vendas"]:
#     base[col] = base[col].fillna(0)

# # flags baseadas na meta
# base["flag_sem_meta"]  = base["meta_n_med"].isna()
# base["flag_meta_zero"] = base["meta_n_med"].fillna(0).eq(0)
# base["flag_meta_pos"]  = base["meta_n_med"].fillna(0).gt(0)

# # flag de venda (aqui uso n_vendas > 0 como “teve venda”; ajuste se quiser faturamento > 0)
# base["flag_teve_venda"] = base["n_vendas"].gt(0)

# # status final (prioridade: SEM_META primeiro)
# base["status"] = np.select(
#     [
#         base["flag_sem_meta"],
#         base["flag_meta_zero"],
#         base["flag_meta_pos"] & (~base["flag_teve_venda"]),
#         base["flag_meta_pos"] & (base["flag_teve_venda"]),
#     ],
#     [
#         "SEM_META",
#         "META_ZERO",
#         "META_POSITIVA_SEM_VENDA",
#         "OK",
#     ],
#     default="?"
# )

# print(base["status"].value_counts())

A = vendas_dia.copy()
B = df_metas_vendas.copy()

# padroniza datas
A["data"] = pd.to_datetime(A["data"]).dt.date
B["data"] = pd.to_datetime(B["data"]).dt.date

# merge
base = A.merge(
    B[["codigo_filial", "data", "meta_n_med"]].drop_duplicates(["codigo_filial", "data"]),
    on=["codigo_filial", "data"],
    how="outer"
)

# completa vendas ausentes com 0
for col in ["quantidade", "faturamento", "n_vendas"]:
    base[col] = base[col].fillna(0)

# cria apenas a coluna status
base["status"] = np.select(
    [
        base["meta_n_med"].isna(),
        base["meta_n_med"].eq(0),
        (base["meta_n_med"] > 0) & (base["faturamento"] == 0),
        (base["meta_n_med"] > 0) & (base["faturamento"] > 0),
    ],
    [
        "SEM_META_NaN",
        "META_ZERO",
        "META_POSITIVA_SEM_VENDA",
        "OK",
    ],
    default="?"
)

print(base["status"].value_counts())


status
OK                         41992
META_ZERO                    627
SEM_META_NaN                  60
META_POSITIVA_SEM_VENDA       34
Name: count, dtype: int64


In [44]:
base

,codigo_filial,data,quantidade,faturamento,n_vendas,meta_n_med,status
0,1500,2025-01-01,0.0,0.00000,0.0,0.00,META_ZERO
1,1500,2025-01-02,828.0,18077.83704,138.0,12732.76,OK
2,1500,2025-01-03,1107.0,29631.50298,188.0,12732.76,OK
3,1500,2025-01-04,597.0,14137.64100,98.0,12732.76,OK
4,1500,2025-01-05,0.0,0.00000,0.0,0.00,META_ZERO
...,...,...,...,...,...,...,...
42708,1887,2025-12-27,597.0,17850.20877,99.0,15986.05,OK
42709,1887,2025-12-28,291.0,9301.56894,62.0,16638.55,OK
42710,1887,2025-12-29,537.0,17140.43700,88.0,16312.30,OK
42711,1887,2025-12-30,735.0,16174.26540,106.0,14681.07,OK


In [ ]:
# # linhas SEM_META (meta_n_med NaN)
# sem_meta = base[base["meta_n_med"].isna()].copy()

# print("Total SEM_META:", len(sem_meta))
# # print("Filiais com SEM_META:", sem_meta["codigo_filial"].nunique())

# # print("\nTop filiais (mais dias sem meta):")
# # print(sem_meta.groupby("codigo_filial").size().sort_values(ascending=False).head(30))

# print("\nAmostra de linhas SEM_META:")
# print(sem_meta.sort_values(["codigo_filial","data"]).head(50))

Total SEM_META: 60

Amostra de linhas SEM_META:
       codigo_filial        data  quantidade   faturamento  n_vendas  \
41734           1863  2025-05-29        12.0       6.15636       4.0   
41735           1863  2025-06-25        15.0       4.55445       5.0   
41736           1863  2025-06-26       537.0   10237.05297      69.0   
41737           1863  2025-06-27       894.0   17364.76722     115.0   
41738           1863  2025-06-28       570.0   12140.21628      78.0   
41739           1863  2025-06-29       279.0    6409.02204      46.0   
41740           1863  2025-06-30       597.0   13523.54409      77.0   
41925           1866  2025-09-17         9.0       2.73267       3.0   
41926           1866  2025-09-25        12.0      85.49802       4.0   
41927           1866  2025-09-26       696.0   10374.84864      65.0   
41928           1866  2025-09-27       390.0   10105.53930      61.0   
41929           1866  2025-09-28       402.0    9653.39235      50.0   
41930           

In [45]:
meta_pos_sem_venda = base[
    (base["meta_n_med"] > 0) & (base["n_vendas"] == 0)
].copy()

#print("Total META_POSITIVA_SEM_VENDA:", len(meta_pos_sem_venda))

#print("\nTop filiais:")
#print(meta_pos_sem_venda.groupby("codigo_filial").size()
#      .sort_values(ascending=False).head(20))

#print("\nTop datas:")
#print(meta_pos_sem_venda.groupby("data").size()
#      .sort_values(ascending=False).head(20))

print("\nAmostra:")
print(meta_pos_sem_venda.sort_values(["codigo_filial","data"]).head(30))



Amostra:
       codigo_filial        data  quantidade  faturamento  n_vendas  \
3549            1527  2025-09-22         0.0          0.0       0.0   
5468            1545  2025-12-25         0.0          0.0       0.0   
5725            1548  2025-09-08         0.0          0.0       0.0   
7293            1560  2025-12-25         0.0          0.0       0.0   
9185            1578  2025-03-02         0.0          0.0       0.0   
9187            1578  2025-03-04         0.0          0.0       0.0   
9483            1578  2025-12-25         0.0          0.0       0.0   
10357           1590  2025-05-18         0.0          0.0       0.0   
10448           1590  2025-08-17         0.0          0.0       0.0   
10511           1590  2025-10-19         0.0          0.0       0.0   
10574           1590  2025-12-21         0.0          0.0       0.0   
13733           1617  2025-08-17         0.0          0.0       0.0   
13740           1617  2025-08-24         0.0          0.0       0.0

Foi identificado também alguns problemas com as metas de vendas, foram identificados metas com NaN, e também há lojas que possuem metas e não tiveram vendas. \\
Para seguir com analise, foi criado uma dummy para filiais que possuem faturamento >= meta (1) e faturamento < meta. No caso, filiais com meta = 0, as linhas foram excluídas. \\


In [46]:
# 1) remove casos meta=0 e faturamento=0
base = base[~((base["meta_n_med"] == 0) & (base["faturamento"] == 0))].copy()

# 2) cria indicador: 1 se bateu meta, 0 se não bateu
# (se meta for NaN, fica NaN)
base["bateu_meta"] = np.where(
    base["meta_n_med"].isna(),
    np.nan,
    (base["faturamento"] >= base["meta_n_med"]).astype(int)
)

print(base["bateu_meta"].value_counts(dropna=False))


bateu_meta
0.0    27234
1.0    14931
NaN       60
Name: count, dtype: int64


In [47]:
base

,codigo_filial,data,quantidade,faturamento,n_vendas,meta_n_med,status,bateu_meta
1,1500,2025-01-02,828.0,18077.83704,138.0,12732.76,OK,1.0
2,1500,2025-01-03,1107.0,29631.50298,188.0,12732.76,OK,1.0
3,1500,2025-01-04,597.0,14137.64100,98.0,12732.76,OK,1.0
5,1500,2025-01-06,975.0,21558.72465,166.0,22143.91,OK,0.0
6,1500,2025-01-07,969.0,25274.68470,165.0,22143.91,OK,1.0
...,...,...,...,...,...,...,...,...
42708,1887,2025-12-27,597.0,17850.20877,99.0,15986.05,OK,1.0
42709,1887,2025-12-28,291.0,9301.56894,62.0,16638.55,OK,0.0
42710,1887,2025-12-29,537.0,17140.43700,88.0,16312.30,OK,1.0
42711,1887,2025-12-30,735.0,16174.26540,106.0,14681.07,OK,1.0


In [ ]:
# VERIFICANDO A META MENSAL - SE A FILIAL BATE OU NÃO META MENSAL
dfm = base.copy()
dfm["data"] = pd.to_datetime(dfm["data"])
dfm["mes"] = dfm["data"].dt.to_period("M").astype(str)

# agrega por filial + mês (soma meta diária e faturamento)
mensal = (
    dfm.dropna(subset=["meta_n_med"])  # remove os SEM_META
       .groupby(["codigo_filial", "mes"], as_index=False)
       .agg(meta_mensal=("meta_n_med", "sum"),
            faturamento_mensal=("faturamento", "sum"))
)

# bateu a meta mensal?
mensal["bateu_meta_mensal"] = (mensal["faturamento_mensal"] >= mensal["meta_mensal"]).astype(int)

#print(mensal.head())
print("\nResumo bateu_meta_mensal:")
print(mensal["bateu_meta_mensal"].value_counts())



Resumo bateu_meta_mensal:
bateu_meta_mensal
0    1014
1     388
Name: count, dtype: int64


### Juntado os data sets após verificação

In [56]:
df_final = base.merge(df_filiais, on="codigo_filial", how="left")

In [60]:
# colunas binárias SIM/NÃO (ajuste se tiver mais)
bin_cols = ["delivery", "panvel_clinic", "estacionamento", "atendimento_24_horas"]

# transforma SIM->1, NÃO->0 (mantém NaN se existir)
for c in bin_cols:
    df_final[c] = df_final[c].astype(str).str.strip().str.upper().replace({"SIM": 1, "NÃO": 0, "NAO": 0})
    df_final[c] = pd.to_numeric(df_final[c], errors="coerce")

In [61]:
df_final

,codigo_filial,data,quantidade,faturamento,n_vendas,meta_n_med,status,bateu_meta,faixa_vida,localidade,uf,tipo_estabelecimento,delivery,metragem_area_venda,panvel_clinic,estacionamento,atendimento_24_horas
0,1500,2025-01-02,828.0,18077.83704,138.0,12732.76,OK,1.0,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,0,309.0388,0,0,0
1,1500,2025-01-03,1107.0,29631.50298,188.0,12732.76,OK,1.0,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,0,309.0388,0,0,0
2,1500,2025-01-04,597.0,14137.64100,98.0,12732.76,OK,1.0,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,0,309.0388,0,0,0
3,1500,2025-01-06,975.0,21558.72465,166.0,22143.91,OK,0.0,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,0,309.0388,0,0,0
4,1500,2025-01-07,969.0,25274.68470,165.0,22143.91,OK,1.0,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,0,309.0388,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42220,1887,2025-12-27,597.0,17850.20877,99.0,15986.05,OK,1.0,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,0,632.4274,1,1,0
42221,1887,2025-12-28,291.0,9301.56894,62.0,16638.55,OK,0.0,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,0,632.4274,1,1,0
42222,1887,2025-12-29,537.0,17140.43700,88.0,16312.30,OK,1.0,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,0,632.4274,1,1,0
42223,1887,2025-12-30,735.0,16174.26540,106.0,14681.07,OK,1.0,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,0,632.4274,1,1,0


In [ ]:
# Checagem 
print(f"Total filiais: {df_final['codigo_filial'].nunique()}")
print(f"Com estacionamento: {(df_final.drop_duplicates('codigo_filial')['estacionamento'] == 1).sum()}")
print(f"Com atendimento 24h: {(df_final.drop_duplicates('codigo_filial')['atendimento_24_horas'] == 1).sum()}")
print(f"Com Panvel Clinic: {(df_final.drop_duplicates('codigo_filial')['panvel_clinic'] == 1).sum()}")
print(f"Com delivery: {(df_final.drop_duplicates('codigo_filial')['delivery'] == 1).sum()}")



Total filiais: 124
Com estacionamento: 111
Com atendimento 24h: 4
Com Panvel Clinic: 87
Com delivery: 38


In [ ]:
filiais_unicas = df_final.drop_duplicates('codigo_filial')

filiais_com_estacionamento = filiais_unicas[filiais_unicas['estacionamento'] == 1]['codigo_filial'].tolist()
print(f"Filiais COM estacionamento ({len(filiais_com_estacionamento)}): {filiais_com_estacionamento}")


# Filiais COM atendimento 24h
filiais_com_24h = filiais_unicas[filiais_unicas['atendimento_24_horas'] == 1]['codigo_filial'].tolist()
print(f"\nFiliais COM atendimento 24h ({len(filiais_com_24h)}): {filiais_com_24h}")

# Filiais COM Panvel Clinic
filiais_com_clinic = filiais_unicas[filiais_unicas['panvel_clinic'] == 1]['codigo_filial'].tolist()
print(f"\nFiliais COM Panvel Clinic ({len(filiais_com_clinic)}): {filiais_com_clinic}")

# Filiais COM delivery
filiais_com_delivery = filiais_unicas[filiais_unicas['delivery'] == 1]['codigo_filial'].tolist()
print(f"\nFiliais COM delivery ({len(filiais_com_delivery)}): {filiais_com_delivery}")

Filiais COM estacionamento (111): [1503, 1506, 1512, 1515, 1518, 1521, 1524, 1527, 1530, 1536, 1539, 1542, 1545, 1551, 1554, 1557, 1560, 1563, 1566, 1569, 1572, 1575, 1581, 1587, 1590, 1593, 1596, 1599, 1602, 1605, 1608, 1611, 1614, 1617, 1623, 1626, 1629, 1635, 1638, 1641, 1644, 1647, 1650, 1653, 1656, 1659, 1665, 1668, 1671, 1674, 1677, 1683, 1686, 1689, 1692, 1698, 1701, 1707, 1710, 1713, 1716, 1719, 1725, 1728, 1731, 1734, 1737, 1740, 1743, 1746, 1749, 1752, 1755, 1758, 1761, 1764, 1767, 1770, 1773, 1782, 1785, 1788, 1791, 1794, 1797, 1800, 1803, 1806, 1809, 1812, 1815, 1824, 1830, 1833, 1836, 1842, 1845, 1848, 1851, 1854, 1857, 1860, 1863, 1866, 1869, 1872, 1875, 1878, 1881, 1884, 1887]

Filiais COM atendimento 24h (4): [1554, 1557, 1605, 1692]

Filiais COM Panvel Clinic (87): [1503, 1509, 1512, 1518, 1524, 1536, 1545, 1557, 1563, 1566, 1590, 1593, 1596, 1599, 1602, 1611, 1614, 1626, 1635, 1638, 1641, 1644, 1653, 1656, 1659, 1662, 1665, 1677, 1683, 1686, 1695, 1698, 1710, 1719, 17

In [69]:
print("\nFaixa de vida:", filiais_unicas['faixa_vida'].value_counts(), sep='\n')
print("\nTipo estabelecimento:", filiais_unicas['tipo_estabelecimento'].value_counts(), sep='\n')


Faixa de vida:
faixa_vida
MAIS DE 3 ANOS    93
MENOS DE 1 ANO    12
ENTRE 1-2 ANOS    11
ENTRE 2-3 ANOS     8
Name: count, dtype: int64

Tipo estabelecimento:
tipo_estabelecimento
BAIRRO          76
CENTRO          38
SHOPPING         6
MALL             3
SUPERMERCADO     1
Name: count, dtype: int64


In [70]:
# 1. ANÁLISE POR TIPO DE ESTABELECIMENTO
print("ANÁLISE POR TIPO DE ESTABELECIMENTO")
print("=" * 40)
tipo_estabelecimento_analise = df_final.groupby('tipo_estabelecimento').agg({
    'n_vendas': 'mean',
    'faturamento': 'mean',
    'quantidade': 'mean',
    'codigo_filial': 'nunique'
}).round(2)

tipo_estabelecimento_analise.columns = ['Média Vendas', 'Média Faturamento', 'Média Quantidade', 'Número de Filiais']
print(tipo_estabelecimento_analise)

ANÁLISE POR TIPO DE ESTABELECIMENTO
                      Média Vendas  Média Faturamento  Média Quantidade  \
tipo_estabelecimento                                                      
BAIRRO                      124.28           26671.78            982.07   
CENTRO                      123.34           26165.47            977.31   
MALL                         77.86           17980.18            872.08   
SHOPPING                    267.85           44799.51           1715.85   
SUPERMERCADO                 72.93           11618.27            477.43   

                      Número de Filiais  
tipo_estabelecimento                     
BAIRRO                               76  
CENTRO                               38  
MALL                                  3  
SHOPPING                              6  
SUPERMERCADO                          1  


In [72]:
# 2. ANÁLISE POR PRESENÇA DE PANVEL CLINIC
print("\n\nANÁLISE POR PANVEL CLINIC")
print("=" * 40)

# garante 0/1 (SIM=1, NÃO=0)
df_final["panvel_clinic"] = (
    df_final["panvel_clinic"].astype(str).str.strip().str.upper()
    .replace({"SIM": 1, "NÃO": 0, "NAO": 0})
)
df_final["panvel_clinic"] = pd.to_numeric(df_final["panvel_clinic"], errors="coerce")

panvel_analise = df_final.groupby("panvel_clinic").agg({
    "n_vendas": "mean",
    "faturamento": "mean",
    "quantidade": "mean",
    "codigo_filial": "nunique"
}).round(2)

panvel_analise.columns = ["Média Vendas", "Média Faturamento", "Média Quantidade", "Número de Filiais"]

# troca o índice só na exibição: 0/1 -> NÃO/SIM
panvel_analise.index = panvel_analise.index.map({0: "NÃO", 1: "SIM"})

print(panvel_analise)




ANÁLISE POR PANVEL CLINIC
               Média Vendas  Média Faturamento  Média Quantidade  \
panvel_clinic                                                      
NÃO                  140.12           29403.74           1055.39   
SIM                  126.11           26311.58            998.19   

               Número de Filiais  
panvel_clinic                     
NÃO                           37  
SIM                           87  


In [73]:
df_final

,codigo_filial,data,quantidade,faturamento,n_vendas,meta_n_med,status,bateu_meta,faixa_vida,localidade,uf,tipo_estabelecimento,delivery,metragem_area_venda,panvel_clinic,estacionamento,atendimento_24_horas
0,1500,2025-01-02,828.0,18077.83704,138.0,12732.76,OK,1.0,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,0,309.0388,0,0,0
1,1500,2025-01-03,1107.0,29631.50298,188.0,12732.76,OK,1.0,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,0,309.0388,0,0,0
2,1500,2025-01-04,597.0,14137.64100,98.0,12732.76,OK,1.0,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,0,309.0388,0,0,0
3,1500,2025-01-06,975.0,21558.72465,166.0,22143.91,OK,0.0,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,0,309.0388,0,0,0
4,1500,2025-01-07,969.0,25274.68470,165.0,22143.91,OK,1.0,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,0,309.0388,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42220,1887,2025-12-27,597.0,17850.20877,99.0,15986.05,OK,1.0,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,0,632.4274,1,1,0
42221,1887,2025-12-28,291.0,9301.56894,62.0,16638.55,OK,0.0,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,0,632.4274,1,1,0
42222,1887,2025-12-29,537.0,17140.43700,88.0,16312.30,OK,1.0,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,0,632.4274,1,1,0
42223,1887,2025-12-30,735.0,16174.26540,106.0,14681.07,OK,1.0,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,0,632.4274,1,1,0
